# Solución al test Teamcore

## Librerías

In [1]:
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd
from unidecode import unidecode

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


## Parte 1
Implementar uno o varios procesos que permitan scrapear la información desde la plataforma (https://www.datosabiertos.gob.pe/). Para descargar el reporte en la opción de **Tipos de Contenido** seleccionar **Dataset**, en **Categoría** seleccionar **Economía y Finanzas** y por último en la opción **formato** seleccionar **csv**. El siguiente paso es buscar en el input **Search** el nombre del reporte en este caso **“donaciones”** y dar click en el botón **consultar**. Luego dar **click** en el reporte **“Donaciones COVID-19 - [Ministerio de Economía y Finanzas - MEF]”**. Finalmente debemos descargar el archivo dando click en el botón **descargar** de la opción **Data - Donaciones Covid-19**.

In [6]:
DOWNLOAD_OUTPUT_DIRECTORY = './data'

class TeamcoreScraper:
    def __init__(self, url):
        self.url = url
        self.driver = webdriver.Firefox(executable_path='./utils/geckodriver')

    def _get_element_by_xpath(self, xpath_string):
        """
        General abstraction to get elements by xpath. Avoiding method redefinition.
        """
        return self.driver.find_element(By.XPATH, xpath_string)

    def _get_elements_by_xpath(self, xpath_string):
        """
        General abstraction to get elements by xpath. Avoiding method redefinition.
        """
        return self.driver.find_elements(By.XPATH, xpath_string)

    def _click_last_selectors(self, last_n):
        """
        Click the last selectors, defined from last to first by last_n.
        """
        selectors_elements = self._get_elements_by_xpath(
            "//h2[@class='pane-title ctools-collapsible-handle']")
        for selector_element in selectors_elements[last_n:]:
            selector_element.click()

    def _get_actual_panel_links(self):
        """
        General abstraction to get actual panel links. Avoiding string redefinition.
        """
        return self._get_elements_by_xpath("//a[@class='facetapi-inactive']")

    def _get_url_by_name(self, name):
        """
        Retrieves each panel link. It will change after picking an option, so running this again is necessary.
        """
        panel_links = self._get_actual_panel_links()
        for panel_link in panel_links:
            link_name = unidecode(panel_link.text.split(
                '\n')[0].split('(')[0].lower().strip()) # Removing unnecesary text and lowercasing
            if link_name == name:
                url = panel_link.get_attribute('href')
                return url

    def _select_option(self, name, click_last_selectors=0):
        """
        General abstraction for option picking.
        """
        if click_last_selectors:
            self._click_last_selectors(click_last_selectors) # To enable the last hidden panel entries
        return self._get_url_by_name(name)

    def _set_selection_criteria(self, content_type, category, data_format):
        """
        Set the three steps in the selection criteria.
        """
        content_type_url = self._select_option(content_type)
        self.driver.get(content_type_url)
        print(f'Added {content_type} as Content Type')

        category_url = self._select_option(category)
        self.driver.get(category_url)
        print(f'Added {category} as Category')

        data_format_url = self._select_option(
            data_format, click_last_selectors=1) # In this case we just need one hidden panel entry
        self.driver.get(data_format_url)
        print(f'Added {data_format} as Data Format')

    def _set_search_term(self, report_name):
        """
        Write the report_name into the search input.
        """
        search_input_element = self._get_element_by_xpath(
            "//input[@class='form-control form-text']")
        search_input_element.send_keys(report_name)
        print(f'Set {report_name} in the search input')
        search_input_element.submit()

    def _execute_custom_behavior(self):
        """
        Click on the single link and downloading the file related to it.
        """
        self.driver.implicitly_wait(1) # Webpage loads slower, so our scraper needs to wait.
        wanted_link_element = self._get_element_by_xpath(
            "//a[@title='Donaciones COVID-19 - [Ministerio de Economía y Finanzas - MEF]']")
        wanted_link_element.click()
        print(f'Clicking on wanted link (defined by end-user)')

        download_button_element = self._get_element_by_xpath(
            "//a[@title='Data - Donaciones Covid-19']/following-sibling::span/a")
        download_button_link = download_button_element.get_attribute('href')
        res = requests.get(download_button_link)
        print(f'Downloading data and saving into {DOWNLOAD_OUTPUT_DIRECTORY}')
        open(f'{DOWNLOAD_OUTPUT_DIRECTORY}/data.zip', 'wb').write(res.content) # We save the file with this method to use the project internal folder structure

    def scrap_data(self, content_type, category, data_format, report_name):
        """
        Extracts data from the url defined in the class.

        This method is divided in three processes:
        - Set the selection criteria by using the content type, the category and data format
        - Set the search term by the report name
        - Execute custom behavior (defined by the end-user)
        
        Parameters
        ----------
        content_type : str
            A string representating the content type
        category : str
            A string representating the category
        data_format : str
            A string representating the data format
        report_name : str
            A string representating the report name
        """
        print(f'Navigating into {self.url}')
        self.driver.get(self.url)
        self._set_selection_criteria(content_type, category, data_format)
        self._set_search_term(report_name)
        self._execute_custom_behavior()
        print(f'Finishing scraping process')

scraper = TeamcoreScraper('https://www.datosabiertos.gob.pe/')
scraper.scrap_data('dataset', 'economia y finanzas', 'csv', 'donaciones')

/tmp/ipykernel_3723273/1858819307.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Firefox(executable_path='./utils/geckodriver')


Navigating into https://www.datosabiertos.gob.pe/
Added dataset as Content Type
Added economia y finanzas as Category
Added csv as Data Format
Set donaciones in the search input
Clicking on wanted link (defined by end-user)
Finishing scraping process


## Parte 2
Crear un proceso que tenga como entrada el nombre del archivo y leerlo con el paquete de pandas pandas. Luego se debe filtrar por la columna REGIÓN, y guardar un archivo con la información de cada región, el nombre del archivo csv debe ser el de la región en minúsculas. Por ejemplo “lima.csv” y la información que contiene debe pertenecer solo de la región Lima.

In [7]:
OUTPUT_DIRECTORY = './region_data'

def create_region_data_by_initial_csv(path_to_file):
    """
    Creates new .csv files for each region by an initial csv file. It accepts compressed files.
    
    Parameters
    ----------
    path_to_file : str
        Path to the initial csv file to be readed
    """
    df = pd.read_csv(path_to_file, encoding='latin-1')
    regions = df['REGION'].unique()
    for region in regions:
        region_df = df[df['REGION'] == region]
        region_df.to_csv(f'{OUTPUT_DIRECTORY}/{region.lower()}.csv', index=False)
        print(f'A csv file for {region} was generated in {OUTPUT_DIRECTORY}...')
create_region_data_by_initial_csv('./data/data.zip')

A csv file for LIMA was generated in ./region_data...
A csv file for LORETO was generated in ./region_data...
A csv file for CAJAMARCA was generated in ./region_data...
A csv file for MADRE DE DIOS was generated in ./region_data...
A csv file for HUANUCO was generated in ./region_data...
A csv file for AMAZONAS was generated in ./region_data...
A csv file for UCAYALI was generated in ./region_data...
A csv file for AREQUIPA was generated in ./region_data...
A csv file for HUANCAVELICA was generated in ./region_data...
A csv file for CUSCO was generated in ./region_data...
A csv file for TACNA was generated in ./region_data...
A csv file for LA LIBERTAD was generated in ./region_data...
A csv file for JUNIN was generated in ./region_data...
A csv file for APURIMAC was generated in ./region_data...
A csv file for AYACUCHO was generated in ./region_data...
A csv file for PROVINCIA CONSTITUCIONAL DEL CALLAO was generated in ./region_data...
A csv file for TUMBES was generated in ./region_d